In [1]:
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

In [ ]:
# create train set
def dist(file):
    fname = list(file.keys())[-1]
    dat = file[fname]
    dist_mat = cdist(dat, dat, 'euclidean')
    return dist_mat[np.triu_indices(78, 1)].flatten()


label = pd.read_csv('../data/train_set/label.csv')
file_path = [os.path.join('../data/train_set/points/', str(i).zfill(4) + '.mat') for i in range(1, 2501)]

names = [f'featrue{n}' for n in range(3003)]
X = pd.DataFrame(list(map(dist, (loadmat(f) for f in file_path))), columns=names)
y = np.array(label['emotion_idx'])
groups = y

train_set = X
train_set['label'] = y

In [ ]:
'''
# Grid Search RF
rf_params = {'n_estimators': [5, 10, 15, 30, 50],
             'max_depth': [5, 10, 15, 20]}

rf_grid = GridSearchCV(RandomForestClassifier(n_jobs=5, verbose=True),
                       rf_params, cv=5, return_train_score=True,
                       scoring='accuracy')
rf_grid.fit(X, y, groups=groups)
print("RandomForest Best Params", rf_grid.best_params_)
print("Score", rf_grid.best_score_)

# Grid Search SVM
svm_params = [{'kernel': ['linear', 'rbf'],
               'C': [1, 10, 50, 100],
               'gamma': [1e-4, 1e-3, 'auto'],
               'decision_function_shape': ['ovo', 'ovr']
               }]
svm_grid = GridSearchCV(SVC(), svm_params, cv=5, verbose=5, n_jobs=5, return_train_score=True, scoring='accuracy')
svm_grid.fit(X, y, groups=groups)
print("SVM best param", svm_grid.best_params_)
print("SVM scores", svm_grid.best_score_)


# Grid Search Logistic
lr_params = {'C': [.1, 1, 10]}
lr_grid = GridSearchCV(LogisticRegression(penalty='l2', max_iter=100),
                       lr_params, cv=5, n_jobs=5, return_train_score=True, verbose=10)
lr_grid.fit(X, y, groups=groups)
print(lr_grid.best_params_)
'''

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True, stratify=y)

rf = RandomForestClassifier(n_estimators=800, max_depth=8, n_jobs=4)
svm = SVC(kernel='linear', C=.001, decision_function_shape='ovo')
svm_bag = BaggingClassifier(svm, n_estimators=120, n_jobs=4)
lr = LogisticRegression(n_jobs=4)
estimators = {'RandomForest': rf, 'SVM': svm, 'SVM with Bagging': svm_bag, 'Logistic Regression': lr}
# Single Score
for name, estimator in estimators.items():
    start = time.time()
    estimator.fit(X_train, y_train)
    print(f'{name}, Train score: {estimator.score(X_train, y_train):.4f}')
    print(f'{name}, Test score: {estimator.score(X_test, y_test):.4f}')
    print(f'{name}, Time spent: {time.time()-start:.4f} seconds')


In [ ]:
# Cross_validation Score
for name, estimator in estimators.items():
    est_cv = cross_validate(estimator, X, y, groups=groups, cv=5, return_train_score=True)
    times = est_cv['fit_time']
    train_scores = est_cv['train_score']
    test_scores = est_cv['test_score']
    print(f'{name}, Mean Train score: {train_scores.mean():.4f}')
    print(f'{name}, Mean Test score: {test_scores.mean():.4f}')
    print(f'{name}, Mean Time spent: {times.mean():.4f} seconds')
